# Text Classification with Bio-med RoBERTa


In [ ]:
#!pip install tensorflow
#!pip install transformers
!pip install datasets
!pip install transformers[torch]
#!pip install wandb
#!pip install text-preprocessing
#!pip install numba
#!pip install scikit-learn

In [ ]:
import pandas as pd
import datasets
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os

## Prepare dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset = 'new'

if dataset == 'new':
    df1 = pd.read_excel('/content/drive/MyDrive/Bio-med Roberta/dataset/01. NSCLC - Sent to CapeStart.xlsx',
                        usecols=['Title', 'Abstract', 'Include/Exclude'], header=0).replace({'Exclude': 0, 'Include':1}).dropna()
    df2 = pd.read_excel('/content/drive/MyDrive/Bio-med Roberta/dataset/09. NSCLC - Sent to CapeStart.xlsx',
                        usecols=['Title', 'Abstract', 'Decision'], header=0).replace({'Exclude': 0, 'Include':1}).dropna().rename(columns={'Decision': 'Include/Exclude'})
    df3 = pd.read_excel('/content/drive/MyDrive/Bio-med Roberta/dataset/12. NSCLC - Sent to CapeStart.xlsx',
                        usecols=['Title', 'Abstract', 'First pass final decision'], header=0).replace(
                        {'E1 - Review/editorial': 0, 'E3 - Study design': 0, 'E4 - Intervention': 0, 'E5 - Disease (non-NSCLC)': 0,
                        'E6 - Population (non-RET+ NSCLC)': 0,'E7 - Animal/in vitro': 0,'I1 - Include clinical': 1,'I2 - Include EE': 1,
                        'I3 - Include HSUV': 1,'I4 - Include cost': 1}).dropna().rename(columns={'First pass final decision': 'Include/Exclude'})
    df4 = pd.read_excel('/content/drive/MyDrive/Bio-med Roberta/dataset/A. NSCLC - Sent to CapeStart.xlsx', header=0,
                        usecols=['Title', 'Abstract', 'Accept or Reject Code']).replace({'Reject': 0, 'Accept':1}).dropna().rename(
                        columns={'Accept or Reject Code': 'Include/Exclude'})
    df  = pd.concat([df1, df2, df3, df4], axis=0)
    df['info'] = df['Title'].astype(str) + df['Abstract'].astype(str)
    df0 = df[df['Include/Exclude']==0]
    df1 = df[df['Include/Exclude']==1]

    """Train Test Split for include 10, exclude 30"""
    df_train_0 = df0.iloc[:30, :]
    df_test_0 = df0.iloc[30:, :]

    df_train_1 = df1.iloc[:10, :]
    df_test_1 = df1.iloc[10:, :]

    df_train = pd.concat([df_train_0, df_train_1], axis=0)
    df_test = pd.concat([df_test_0, df_test_1], axis=0)


elif dataset == 'old':
    df_train = pd.read_csv('/content/drive/MyDrive/Bio-med Roberta/dataset/hpv1_train.csv', header=0,
                           usecols=['title', 'abstract', 'level1_labels']).dropna().rename(
                           columns={'title': 'Title', 'abstract': 'Abstract', 'level1_labels': 'Include/Exclude'})
    df_train['info'] = df_train['Title'].astype(str) + df_train['Abstract'].astype(str)
    X_train = df_train['info'].to_numpy().reshape(-1)
    y_train = df_train['Include/Exclude'].to_numpy().reshape(-1)

    df_test = pd.read_csv('/content/drive/MyDrive/Bio-med Roberta/dataset/hpv1_test.csv', header=0,
                          usecols=['title', 'abstract', 'level1_labels']).dropna().rename(
                          columns={'title': 'Title', 'abstract': 'Abstract', 'level1_labels': 'Include/Exclude'})
    df_test['info'] = df_test['Title'].astype(str) + df_test['Abstract'].astype(str)
    X_test = df_test['info'].to_numpy().reshape(-1)
    y_test = df_test['Include/Exclude'].to_numpy().reshape(-1)

    # print(df_train.head(5))

# Train Split

In [ ]:
#print(df1.shape, df2.shape, df.shape)

(200, 4) (195, 4) (395, 4)


In [ ]:
if dataset=='new':
    X_train = df_train['info'].to_numpy().reshape(-1)
    y_train = df_train['Include/Exclude'].to_numpy().reshape(-1)
    X_test = df_test['info'].to_numpy().reshape(-1)
    y_test = df_test['Include/Exclude'].to_numpy().reshape(-1)

else:
    X_train, X_test = np.append(X_data[:30], X_data[200:230], axis=0), np.append(X_data[30:200], X_data[230:], axis=0)
    y_train, y_test = np.append(y_data[:30], y_data[200:230], axis=0), np.append(y_data[30:200], y_data[230:], axis=0)

X_train = X_train.reshape((-1,)).tolist()
X_test = X_test.reshape((-1,)).tolist()

In [ ]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Load Model

In [ ]:
MODEL_NAME = 'allenai/biomed_roberta_base'
MAX_LEN = 512
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME, max_length = MAX_LEN)

# Tokenize & encode

In [ ]:
def roberta_encoder(tokenizer, data, label):


  encoded_inputs = tokenizer(data, padding=True, truncation=True, return_tensors='pt')

  dataset_dict = {
    'input_ids': encoded_inputs['input_ids'],
    'attention_mask': encoded_inputs['attention_mask'],
    'label': torch.tensor(label)}

  dataset = datasets.Dataset.from_dict(dataset_dict)

  # You can also set the format explicitly
  #dataset = dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

  return dataset


train_data = roberta_encoder(tokenizer, X_train, y_train)
test_data = roberta_encoder(tokenizer, X_test, y_test)

# Compute Metrics

In [ ]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    report = classification_report(labels, preds, output_dict=True)

    precision_0, recall_0, f1_0, _ = report['0'].values()
    precision_1, recall_1, f1_1, _ = report['1'].values()

    ad_hoc_1 = recall_1 * .8 + precision_1 * .2
    acc = accuracy_score(labels, preds)
    return {
            'accuracy': acc,
            'f1': f1,
            'precision_0': precision_0,
            'recall_0': recall_0,
            'precision_1': precision_1,
            'recall_1': recall_1,
            'adhoc_1': ad_hoc_1
            }

# Training Arguements

In [ ]:
training_args = TrainingArguments(
                                  output_dir = '/content/drive/MyDrive/Bio-med Roberta',
                                  overwrite_output_dir= True,
                                  num_train_epochs=10,
                                  per_device_train_batch_size = 15,
                                  # gradient_accumulation_steps = 16,
                                  per_device_eval_batch_size= 15,
                                  evaluation_strategy = "epoch",
                                  save_strategy = 'epoch',
                                  disable_tqdm = False,
                                  load_best_model_at_end=True,
                                  warmup_steps=500,
                                  weight_decay=0.01,
                                  logging_steps = 8,
                                  fp16 = False,
                                  report_to='none',
                                  save_total_limit=1,
                                  # logging_dir='/media/jlealtru/data_files/github/website_tutorials/logs',
                                  # dataloader_num_workers = 8,
                                  # run_name = 'roberta-classification'
                                  )


trainer = Trainer(
                  model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=train_data,
                  eval_dataset=test_data
                  )

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Train model

In [ ]:
trainer.train(resume_from_checkpoint=False)

In [ ]:
trainer.eval()

relevant=10
irrelevant=30

In [ ]:
res = trainer.predict(test_data)
print(classification_report(res.label_ids, np.argmax(res.predictions, axis=1)))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      3860
           1       0.08      0.03      0.04       185

    accuracy                           0.94      4045
   macro avg       0.52      0.51      0.51      4045
weighted avg       0.92      0.94      0.93      4045



## Evaluation

In a confusion matrix, we can see how many categories are classified c

In [ ]:
#print(classification_report(res.label_ids, np.argmax(res.predictions, axis=1)))
print(precision_recall_fscore_support(res.label_ids, np.argmax(res.predictions, axis=1), average='binary'))

(0.0847457627118644, 0.02702702702702703, 0.04098360655737705, None)
